## Energy cascade mechanism in turbulence using a shell model



## Parameters

- `PowMax`: The maximum order of the structure function.
- `N`: The number of shells in the GOY model. 
- `lambda`: The intershell ratio, usually set to 2.0. 
- `k0`: The wavenumber at n = 0. This is set to 0.05 
- `delta`: The GOY parameter in the nonlinear interaction term. 
- `nu`: The viscosity. This is set to 1.0e-05 
- `DT`: The time step for the numerical integration. 
- `Nstat`: The number of time steps. This should be high enough for good statistics. This is set to 10^7 
- `update_dt`: The number of time steps for updating `dt`. This is set to 10^5
- `decay_mode`: If set to 1, the forcing is stopped after a given number of time steps. This is set to 0 in this script, meaning the forcing is not stopped.
- `img`: The imaginary unit. This is set to `complex(0.,1.)` 
- `forc`: The forcing term. This is set to `complex(1.e-02,1.e-02)` 

In [2]:
def goy_rhs(u, k, N, delta, lambda_, forc, img):
    coeffb = -delta / lambda_
    coeffc = -(1.0 - delta) / (lambda_**2)

    cu = np.conj(u)

    du = np.zeros(N, dtype=complex)
    du[0] = img*k[0]*(cu[1]*cu[2])
    du[1] = img*k[1]*(cu[2]*cu[3]+coeffb*cu[0]*cu[2])

    aa = cu[3:N-1]*cu[4:N]
    bb = cu[1:N-3]*cu[3:N-1]
    cc = cu[0:N-4]*cu[1:N-3]
    du[2:N-2] = img*k[2:N-2]*(aa+coeffb*bb+coeffc*cc)

    du[N-2] = img*k[N-2]*(coeffb*cu[N-3]*cu[N-1] + coeffc*cu[N-4]*cu[N-3])
    du[N-1] = img*k[N-1]*(coeffc*cu[N-3]*cu[N-2])

    # Force lowest shell
    du[0] += forc

    return du

In [3]:
import numpy as np

# Global variables
PowMax = 10
N = 20
lambda_ = 2.0
k0 = 0.05
delta = 0.5
nu = 1.0e-05
DT = 0.001
Nstat = 10**7
update_dt = 10**5
decay_mode = 0

img = complex(0.,1.)
forc = complex(1.e-02,1.e-02)

Stat = 0
dt = DT

# Compute wavenumbers
k = k0 * lambda_ ** np.arange(N)

# Integrating factors for the viscous term
f1 = np.array([np.exp(-0.5 * nu * dt * k[j]**2) for j in range(N)], dtype=complex)
f2 = (1 - f1) / (nu * k**2)

# Set the average velocity, mu, to zero
mu = np.zeros(N)

# Initialize the velocity according to Kolmogorov scaling
stat = 0
dtmp = (k / k[0]) ** (-1/3)
u1 = np.array([complex(d, 0.0) for d in dtmp])
du1 = goy_rhs(u1, k, N, delta, lambda_, forc, img)

# Working velocity
u = u1.copy()

# Initialize averaged structure function and energies
Sp = np.zeros((PowMax, N))
eps_av = 0
en_av = 0
eps_forc = 0

# Init time
t = 0.0

# Main integration loop
for stat in range(Nstat):
    mu += np.abs(u)
    t += dt
    
    if stat % update_dt == 0 and stat > 0:
        dtmp = 1.0 / (k * mu / update_dt)
        dt = np.min(dtmp) / 50.0
        f1 = np.array([np.exp(-0.5 * nu * dt * k[j]**2) for j in range(N)], dtype=complex)
        f2 = (1 - f1) / (nu * k**2)
        mu = np.zeros(N)
        
        # Write to file
        with open("tau.dat", "w") as fout:
            for i in range(N):
                fout.write(f"{i} {k[i]:e} {dtmp[i]:e}\n")
        print(f"Update at stat {stat}: dt = {dt:e}")
    
    # Integration step
    du2 = goy_rhs(u, k, N, delta, lambda_, forc, img)
    u2 = f1 * u + f2 * (1.5 * du2 - 0.5 * du1)
    du1 = du2
    u = u2.copy()
    
    # Decay mode
    if stat > 10**5 and decay_mode:
        forc = 0
        print(f"\nStopped forcing at t: {t:e}\n")

Update at stat 100000: dt = 1.074516e-03
Update at stat 200000: dt = 9.953066e-04
Update at stat 300000: dt = 1.342376e-03
Update at stat 400000: dt = 1.509072e-03
Update at stat 500000: dt = 1.172315e-03
Update at stat 600000: dt = 1.156572e-03
Update at stat 700000: dt = 9.568170e-04
Update at stat 800000: dt = 1.123727e-03
Update at stat 900000: dt = 1.280235e-03
Update at stat 1000000: dt = 1.051172e-03
Update at stat 1100000: dt = 1.215196e-03
Update at stat 1200000: dt = 1.396950e-03
Update at stat 1300000: dt = 2.695733e-03
Update at stat 1400000: dt = 1.378293e-03
Update at stat 1500000: dt = 2.551703e-03
Update at stat 1600000: dt = 1.008876e-03
Update at stat 1700000: dt = 8.400436e-04
Update at stat 1800000: dt = 8.462098e-04
Update at stat 1900000: dt = 1.408128e-03
Update at stat 2000000: dt = 2.114942e-03
Update at stat 2100000: dt = 8.140813e-04
Update at stat 2200000: dt = 1.631254e-03
Update at stat 2300000: dt = 1.853270e-03
Update at stat 2400000: dt = 1.204940e-03
U